In [17]:
import torch,sys,os
from torch import nn
from tqdm import tqdm
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from argparse import ArgumentParser
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if "__file__" in globals():
    main_folder_path = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
else:
    main_folder_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(main_folder_path)
from baseclasses.baseclass import BaseTrainer
from architectures.mlpRegressor import MLP_Regressor
from dataloaders.dataloaders import WineQualityLoader
from trainers.trainer_mlpRegressor import MLPRegressorTrainer
from algorithms.feature_CP import ConformalPredictionMethods
from algorithms.conformal import Conformal 

In [62]:
red_wine= pd.read_csv('datasets/winequality-red.csv', sep=';')
white_wine = pd.read_csv('datasets/winequality-white.csv', sep=';')
red_wine['type'] = 'red'
white_wine['type'] = 'white'
wines = pd.concat([red_wine, white_wine], ignore_index=True)
X = wines.iloc[:, :-2].to_numpy()
y= wines.iloc[:, -2].to_numpy()
N = X.shape[0] # number of observations
T = 10
cov = np.zeros(T)
length = np.zeros(T)
# X.shape,y.shape,X[0],N,y
# BIAS= np.random.randn(X.shape[1],X.shape[1])
# BIAS_LEFT = np.random.randn(X.shape[0],X.shape[0])
# X_norm = X @ BIAS
# X_norm = BIAS_LEFT @ X_norm
# X = X_norm
# X_norm.shape,X_norm[0],X[0]

In [64]:
for i in tqdm(range(T)):
    if (i + 1) % 20 == 0:
            print(i + 1, "..", sep="")
    idx = np.random.choice(range(0, N), size=int(N * 0.5), replace=False)
    noidx = np.setdiff1d(range(0, N), idx)

    X0 = X[idx, :] # training set
    y0 = y[idx]
    X00 = X[noidx, :] # calibration set 
    y00 = y[noidx]
    n00 = X00.shape[0] # size of calibration set
    Split_CP_CCS = Conformal(X0=X0, y0=y0, X00=X00, y00=y00, rho=0.5, alpha=0.1, weight=None, train_model="lasso")
    cov[i] = np.mean(Split_CP_CCS.split_weight_conformal()[0])
    length[i] = np.median(Split_CP_CCS.split_weight_conformal()[1])
np.mean(cov),np.mean(length),np.std(cov),np.std(length)

100%|██████████| 10/10 [00:09<00:00,  1.03it/s]


(0.8994152046783626,
 2.4189084451550755,
 0.008816687744488941,
 0.01743191384063448)

In [67]:
X00.shape,X0.shape,torch.tensor(y0).unsqueeze(-1).shape,y00.shape

((3249, 11), (3248, 11), torch.Size([3248, 1]), (3249,))